In [1]:
using Pkg
Pkg.activate("Dev")
using Skylight
using BenchmarkTools

  Activating project at `~/repos/Skylight.jl/run/Dev`


In [11]:
function time_intensities(; N, chunks_per_thread)
    spacetime = SchwarzschildSpacetimeSphericalCoordinates(M=1.0)
    camera = ImagePlane(distance = 200.0,
                            observer_inclination_in_degrees = 5,
                            horizontal_side = 20.0,
                            vertical_side = 20.0,
                            horizontal_number_of_pixels = N,
                            vertical_number_of_pixels = N)
    model = NovikovThorneDisk(inner_radius=6.0, outer_radius=18.0)
    configurations = VacuumOTEConfigurations(spacetime=spacetime,
                                    camera = camera,
                                    radiative_model = model,
                                    unit_mass_in_solar_masses=1.0)
    initial_data = initialize(configurations)
    cb, cbp = callback_setup(configurations; rhorizon_bound=2e-1) #... or, define your own cb and cbp
    run = integrate(initial_data, configurations, cb, cbp; method=VCABM(), reltol=1e-5, abstol=1e-5)
    output_data = run.output_data
    println("Serial")
    @time _ = Skylight.observed_bolometric_intensities_serial(initial_data, output_data, configurations, camera)
    println("Multithread")
    @time _ = observed_bolometric_intensities(initial_data, output_data, configurations, camera; chunks_per_thread=chunks_per_thread)
    return nothing
end
println("compile funcs")
time_intensities(N=3, chunks_per_thread=2)

compile funcs


Equations integration stats:
Wall clock time: 0.001534181 seconds
Memory allocated: 323.258 KiB
Serial
  0.000010 seconds (11 allocations: 1.234 KiB)
Multithread
  0.000071 seconds (114 allocations: 13.266 KiB)


In [14]:
function btime_intensities(; N, chunks_per_thread)
    spacetime = SchwarzschildSpacetimeSphericalCoordinates(M=1.0)
    camera = ImagePlane(distance = 200.0,
                            observer_inclination_in_degrees = 5,
                            horizontal_side = 20.0,
                            vertical_side = 20.0,
                            horizontal_number_of_pixels = N,
                            vertical_number_of_pixels = N)
    model = NovikovThorneDisk(inner_radius=6.0, outer_radius=18.0)
    configurations = VacuumOTEConfigurations(spacetime=spacetime,
                                    camera = camera,
                                    radiative_model = model,
                                    unit_mass_in_solar_masses=1.0)
    initial_data = initialize(configurations)
    cb, cbp = callback_setup(configurations; rhorizon_bound=2e-1) #... or, define your own cb and cbp
    run = integrate(initial_data, configurations, cb, cbp; method=VCABM(), reltol=1e-5, abstol=1e-5)
    output_data = run.output_data
    println("Serial")
    @btime _ = Skylight.observed_bolometric_intensities_serial($initial_data, $output_data, $configurations, $camera)
    println("Multithread")
    @btime _ = observed_bolometric_intensities($initial_data, $output_data, $configurations, $camera; chunks_per_thread = $chunks_per_thread)
    return nothing
end
println("compile funcs")
time_intensities(N=3, chunks_per_thread=2)

compile funcs
Equations integration stats:
Wall clock time: 0.001407017 seconds
Memory allocated: 320.062 KiB
Serial
  0.000007 seconds (11 allocations: 1.234 KiB)
Multithread
  0.001697 seconds (119 allocations: 13.125 KiB)


In [15]:
time_intensities(N=3, chunks_per_thread=2)

Equations integration stats:
Wall clock time: 0.002705067 seconds
Memory allocated: 318.938 KiB
Serial
  0.000005 seconds (11 allocations: 1.234 KiB)
Multithread
  0.000052 seconds (112 allocations: 12.906 KiB)


In [16]:
btime_intensities(N=3, chunks_per_thread=2)

Equations integration stats:
Wall clock time: 0.000822085 seconds
Memory allocated: 319.062 KiB
Serial
  

1.435 μs (11 allocations: 1.23 KiB)
Multithread


  7.684 μs (110 allocations: 12.84 KiB)


In [18]:
btime_intensities(N=10, chunks_per_thread=2)

Equations integration stats:
Wall clock time: 0.018777604 seconds
Memory allocated: 3.066 MiB
Serial
  

19.911 μs (73 allocations: 8.55 KiB)
Multithread


  17.910 μs (172 allocations: 20.16 KiB)


In [20]:
btime_intensities(N=50, chunks_per_thread=2)

Equations integration stats:
Wall clock time: 0.239531949 seconds
Memory allocated: 78.439 MiB
Serial
  

477.916 μs (1601 allocations: 189.08 KiB)
Multithread


  272.402 μs (1700 allocations: 200.69 KiB)


In [21]:
btime_intensities(N=50, chunks_per_thread=1) 

Equations integration stats:
Wall clock time: 0.362930422 seconds
Memory allocated: 78.439 MiB
Serial
  

478.374 μs (1601 allocations: 189.08 KiB)
Multithread


  262.258 μs (1649 allocations: 194.06 KiB)


In [23]:
btime_intensities(N=100, chunks_per_thread=2) 

Equations integration stats:
Wall clock time: 0.905665721 seconds
Memory allocated: 311.537 MiB
Serial
  

1.937 ms (6423 allocations: 758.27 KiB)
Multithread


  985.560 μs (6512 allocations: 768.31 KiB)


In [24]:
time_intensities(N=600, chunks_per_thread=2) 

Equations integration stats:
Wall clock time: 31.703497095 seconds
Memory allocated: 10.995 GiB
Serial
  0.100310 seconds (230.40 k allocations: 26.587 MiB)


Multithread
  0.061184 seconds (230.49 k allocations: 26.597 MiB)


In [25]:
time_intensities(N=1000, chunks_per_thread=2)

Equations integration stats:
Wall clock time: 84.784255883 seconds
Memory allocated: 30.552 GiB
Serial
  0.315176 seconds (639.83 k allocations: 73.837 MiB, 9.91% gc time)


Multithread


  0.219343 seconds (639.98 k allocations: 73.851 MiB, 4.64% gc time)
